# 加载库函数及导入文件

In [7]:
import pathlib
import pandas as pd
output_dir_path = pathlib.Path(r"E:\张楠\测试\output")
print(output_dir_path)

E:\张楠\测试\output


In [202]:
%run corelibs.py

# 测试

## 提取excel表头

In [206]:
%%time
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_16_1_watermark.xlsx")
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\建行第二次查询7.13\1080861701689234045070165_1_1田桂芬.xlsx")
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217号.xls")
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217.xls")
print(a := read_header(acc_file))
hash(a)

('所属银行', '证件号码', '开户名称', '客户账号', '卡号', '余额', '开户日期', '开户机构名称', '是否开通网易', 'U盾号', '短信捆绑电话', '网银电话', '地址', '邮编', '销户日期', '销户机构号', '销户机构名称', '账户状态')
Wall time: 69 ms


-7657799188243513271

## 测试yaml配置文件语法

In [101]:
import yaml
with open('config.yaml', 'r', encoding='utf-8') as f:
    CONF_DATA = yaml.safe_load(f)

In [ ]:
CONF_DATA

## 测试配置转换内部操作表示功能

In [116]:
%run corelibs.py
conf_obj = creat_conf_obj(CONF_DATA['建设银行网点']['流水'])

NameError: name 'Any' is not defined

In [24]:
conf_obj

Conf_tpl(new_cols={'银行': '建设银行'}, verify_cols={'客户名称': True, '交易卡号': True, '账户余额': True, '借贷方向': True, '交易金额': True, '交易日期': True, '交易时间': True}, col_name_map={'客户名称': '姓名', '交易卡号': '卡号', '扩充备注': '备注', '账户余额': '余额', '交易机构名称': '交易机构', '对方行名': '对方开户行'}, merge_2cols={}, date_cols={'日期': '交易日期'}, time_cols={'时间': '交易时间'}, digi_cols={'账户余额': True, '交易金额': True}, cdid={'C': '借', 'CD_col': '借贷方向', 'C_col': '出账金额', 'trans_col': '交易金额', 'D': '贷', 'D_col': '入账金额'}, fill_cols={}, cols_new_order=['银行', '日期', '时间', '姓名', '卡号', '备注', '摘要', '币种', '出账金额', '入账金额', '余额', '交易机构', '对方户名', '对方账号', '对方开户行', 'IP地址', 'MAC地址', '商户名称', '商户号', '交易流水号', '交易渠道', '自助设备编号', '第三方订单号', '账号', '钞汇', '册号', '笔号', '活期存款账户明细号', '借贷方向', '交易金额', '柜员号', '交易机构号'])

##  测试单个账户文件读取及储存

In [106]:
%run corelibs.py
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_1_1赵益华.xlsx")
conf_data = CONF_DATA['建设银行网点']['账户']
conf_obj = creat_conf_obj(conf_data)
df = parse_sheet_general(acc_file, conf_obj)

D:\soft\WPy64-3890\python-3.8.9.amd64\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning:

Workbook contains no default style, apply openpyxl's default



In [54]:
save_accounts(df, output_dir_path, conf_data['银行'])

8

In [ ]:
import dtale

In [108]:
dtale.show(df, open_browser=True)

## 测试单个流水文件的读取与储存

In [25]:
%run corelibs.py
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_16_1_watermark.xlsx")
conf_data = CONF_DATA['建设银行网点']['流水']
# acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217号.xls")
# conf_data = CONF_DATA['天津银行']['流水']
conf_obj = creat_conf_obj(conf_data)
df = parse_sheet_general(acc_file, conf_obj)

In [28]:
grouped = df.groupby('账号')
df_list = [x.reset_index(drop=True) for _ , x in grouped]
save_statements(df_list, output_dir_path, conf_data['银行'],"津滨查询(2023)139号")

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.47s/it]


9999

In [ ]:
import dtale

In [27]:
dtale.show(df, open_browser=True)

2024-01-22 12:17:50,098 - INFO     - NumExpr defaulting to 4 threads.


# 执行

## 分析建设银行网点数据目录
（如：开发区金融街建行查询数据）

In [2]:
from collections import Counter

base_dir = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人")

if base_dir.is_dir():
    _file_names = base_dir.rglob('[!~]*.xlsx') # 找到目录中所有的excel文件
     # 检查是否存在超过9999条的数据（文件名第二段的数字一样）
    _d, _c = Counter(map(lambda x: x.name.split('_')[1], _file_names)).most_common(1)[0]
    if _c > 1: raise Exception(f'目录中包含超过9999条的数据文件，需要手动合并，文件名第二字段为{_d}')
else:
    raise Exception("传入的路径不是目录，请传入目录路径，或使用单文件分析") 
_d, _c

Exception: 目录中包含超过9999条的数据文件，需要手动合并，文件名第二字段为16

# 杂项

In [48]:
pd.read_excel

'7'

In [6]:
hash(str(['所属银行','证件号码','开户名称','客户账号','卡号','余额','开户日期','开户机构名称','是否开通网易','U盾号','短信捆绑电话','网银电话','地址','邮编','销户日期','销户机构号','销户机构名称','账户状态','我行账号（卡号）','我方银行名称','我方中文名称','身份证号','交易日期','对方账号（卡号）','对方中文名称','对方银行名称','对方银行机构代码','流水号','借贷标志','交易金额','币种','余额','网银IP地址','交易代码（交易渠道）','代码描述','用途（摘要）','受理机构代码','受理机构名称','交易是否成功']))

9091559758436992767

In [124]:
type(CONF_DATA['天津银行']['流水']['时间']['交易日期'])

int

In [ ]:
CONF_DATA['天津银行']['流水']

In [27]:
print(CONF_DATA.get('哈哈'))

None


In [37]:
from itertools import islice
a = [3,4,5,6,7,8]
b = islice(a,4,5)
next(b)

7